# Sub-Holding Keys

This notebook demonstrates LUSID's [Sub-holding Keys](https://support.finbourne.com/what-are-subholding-keys) (or SHKs). The core idea with `Sub-holding Keys` - they allow you to bucket your `holding` in one instrument (or [LUID](https://support.finbourne.com/what-is-a-lusid-unique-identifier-luid)) into different groups. For example, in this notebook we have a `Sub-Holding Key` of <i>strategy</i> which is used to tag transactions on the same instrument following different investment strategies. Then in the `holdings` report you can see the position split-out into two buckets. However, this is just one sample implementation of `Sub-Holding Keys`. You are allowed use <u>any</u> pre-defined transaction property as a `Sub-Holding Key`. 

### Setup LUSID

In [1]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

Load a mapping file for DataFrame headers for the `build transaction` and `get holdings` response.

In [2]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

Define our transaction portfolios API

In [3]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

### 1) Declare a scope and load our CSV file

In [4]:
# Create a new scope

scope = "notebook_shk1"
portfolio_code = "EQUITY_UK" + "_" + create_scope_id().replace("-", "")

In [5]:
# Load a file of equity transactions

transactions_file = r"data/shk/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code
transactions_df.tail(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
20,EQUITY_UK_386ac11efa1236,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,ftse_tracker,GBP
21,EQUITY_UK_386ac11efa1236,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_002,FundsIn,02/02/2020,04/01/2020,100000,1,100000,GBP,food_retail,GBP


### 2) Create a property for the new Sub-Holding key

The <b>strategy</b> will be used to create our `Sub-Holding Key` on the portfolio.

In [6]:
domain = "Transaction"
scope = scope
prop_code = "strategy"

try:
    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(

        lusid.models.CreatePropertyDefinitionRequest(

            domain = domain,
            scope = scope,
            code = prop_code,
            value_required = None,
            display_name = "Investment strategy",
            data_type_id = lusid.ResourceId(scope="system", code="string"),
            life_time = None

        )
    )

except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/notebook_shk1/strategy' because it already exists.


### 3) Load default transactions into a new scope

The portfolio is created with the new `Sub-holding Key`.

In [7]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_df,
    "DataFrame",
    sub_holding_keys=[f"Transaction/{scope}/strategy"]
)

### 4) Lets check our holdings

We have can see that the one Tesco instrument (with the same LUID) is bucketed under two different `Sub-Holding Keys` with the <b>strategy</b> label. There is also a seperate CCY_GBP cash line for tracking the cash in each SHK.

In [8]:
response = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df[(holdings_df["Name(default-Properties)"] == 'Tesco') | (holdings_df["Name(default-Properties)"] == 'CCY_GBP') ]

,LusidInstrumentId,strategy(notebook_shk1-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
7,LUID_99M6G8U7,ftse_tracker,Tesco,notebook_shk1/EQUITY_UK_386ac11efa1236,P,8000.0,8000.0,64000.0,GBP,0.0,GBP
8,LUID_99M6G8U7,food_retail,Tesco,notebook_shk1/EQUITY_UK_386ac11efa1236,P,4000.0,4000.0,36000.0,GBP,0.0,GBP
12,CCY_GBP,ftse_tracker,CCY_GBP,notebook_shk1/EQUITY_UK_386ac11efa1236,B,3656000.0,3656000.0,3656000.0,GBP,0.0,GBP
13,CCY_GBP,food_retail,CCY_GBP,notebook_shk1/EQUITY_UK_386ac11efa1236,B,-296000.0,-296000.0,-296000.0,GBP,0.0,GBP
